In [1]:
import os
import gc
import time

import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, GradientBoostingRegressor
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

from typing import Union

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
import os
import gc
import warnings

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from multiprocessing import cpu_count
from joblib import Parallel, parallel_backend
from joblib import delayed
import multiprocessing

import random

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
SEED = 42 

In [2]:
# define random hyperparammeters
# params = {
#     'boosting_type': 'gbdt',
#     'metric': 'rmse',
#     'objective': 'regression',
#     'n_jobs': -1,
#     'seed': 236,
#     'learning_rate': 0.1,
#     'bagging_fraction': 0.75,
#     'bagging_freq': 10, 
#     'colsample_bytree': 0.75}

# # define random hyperparammeters

# params = { 'boosting_type': 'gbdt',
#                     'objective': 'tweedie',
#                     'tweedie_variance_power': 1.1,
#                     'metric': 'rmse',
#                     'subsample': 0.5,
#                     'subsample_freq': 1,
#                     'learning_rate': 0.05,
#                     'num_leaves': 2**11-1,
#                     'min_data_in_leaf': 2**12-1,
#                     'feature_fraction': 0.5,
#                     'max_bin': 100,
#                     'n_estimators': 1400,
#                     'boost_from_average': False,
#                     'verbose': -1} 

      
# params = {
#     'boosting_type': 'gbdt',
#     'n_jobs': -1,
#     'objective': 'poisson',
#     'metric': 'rmse',
#     'seed': 42,
#     'learning_rate': 0.05,
#     'bagging_fraction': 0.85,
#     'bagging_freq': 1,
#     'colsample_bytree': 0.85,
#     'colsample_bynode': 0.85,
#     'min_data_per_leaf': 25,
#     'n_estimators': 1400,
#     'lambda_l1': 0.5,
#     'lambda_l2': 0.5}



# params = {
#         "objective" : "poisson",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
# #         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
# #         "nthread" : 4
#         "metric": ["rmse"],
#     'verbosity': 1,
#     'num_iterations' : 1200,
#     'num_leaves': 128,
#     "min_data_in_leaf": 100,
# }





params = {
                    'boosting_type': 'gbdt',
                    'objective': 'regression',
                    'metric': ['rmse'],           
                    'subsample': 0.5,                
                    'subsample_freq': 1,
                    'learning_rate': 0.05,           
                    'num_leaves': 2**8-1,            
                    'min_data_in_leaf': 2**8-1,     
                    'feature_fraction': 0.8,
                    'n_estimators': 1500,            
                    'early_stopping_rounds': 30,     
                    'seed': SEED,
                    'verbose': -1,
                } 



# params = {
#         "objective" : "poisson",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
# #         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
# #         "nthread" : 4
#         "metric": ["rmse"],
#     'verbosity': 1,
#     'num_iterations' : 1200,
#     'num_leaves': 128,
#     'seed': SEED,
#     "min_data_in_leaf": 100,
# }



In [3]:
# import pickle

# with open("../input/1-stats-id/1_list_cal_CA.pkl", "rb") as x1:
#     X1 = x1.read()
    
# X_ca_1 = pickle.loads(X1)
# pd.DataFrame(X_ca_1[0]).iloc[:-28,:]

## STORE

In [4]:
import pickle

with open("../input/1-stats-id/1_X_CA.pkl", "rb") as x1:
    X1 = x1.read()
    
with open("../input/1-2-stats-id/2_X_CA.pkl", "rb") as x1_1:
    X1_1 = x1_1.read()
    
del x1, x1_1 
gc.collect()

X_ca_1 = pickle.loads(X1)
X_ca_2 = pickle.loads(X1_1)

X_ca = [X_ca_1, X_ca_2]

del X1, X1_1, X_ca_1, X_ca_2 
gc.collect()

0

In [5]:
i=0

for xi in X_ca:
    
    print(xi.shape)
    
    STORES = np.unique(xi[:,3])
    print('')
    print('number of STORES')
    print(STORES)
    print('')

    for store in STORES:

        print('')
        print('store '+str(store))
        print('')
        xi_store = xi[xi[:,3]==store]
        

        #18 -day
        #19 -month
        #20 - year

        mask_XYtrain = np.logical_or(xi_store[:,21]<=24,xi_store[:,22]<=4,xi_store[:,23]<=2016)

        # <= '2016-03-27' ~
        XY_train =  xi_store#[mask_XYtrain]
        XY_val =  xi_store[~mask_XYtrain]

#         XY_train = np.delete(XY_train,[18,19,20],1)
        XY_train = np.delete(XY_train,[3,4],1)
        XY_val = np.delete(XY_val,[3,4],1)

        X_train = XY_train[:,:-1]
        y_train = XY_train[:,-1]
        X_val = XY_val[:,:-1]
        y_val = XY_val[:,-1]

        del xi_store, XY_train, XY_val
        gc.collect()

        train_set = lgb.Dataset(X_train, y_train.reshape(-1))
        val_set = lgb.Dataset(X_val, y_val.reshape(-1))

        seed_everything(SEED)
        model_ca = lgb.train(params, train_set,
                           valid_sets = [val_set],
#                           num_boost_round = 2500,
#                           early_stopping_rounds = 60,
                          verbose_eval = 100)

    #         model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 500, 
    #                           valid_sets = [train_set, val_set], verbose_eval = 50)

        model_ca.save_model('model_ca_'+str(i)+'.txt')
        print('save model_'+str(i)+'_'+str(int(store)))
        print('')

        del train_set, val_set, model_ca
        gc.collect()

        i+=1
        
    del xi
    gc.collect()

del X_ca
gc.collect()

(10573932, 65)

number of STORES
[0. 1.]


store 0.0

Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 2.24251
[200]	valid_0's rmse: 2.18811
[300]	valid_0's rmse: 2.14389
[400]	valid_0's rmse: 2.10562
[500]	valid_0's rmse: 2.07145
[600]	valid_0's rmse: 2.04096
[700]	valid_0's rmse: 2.01311
[800]	valid_0's rmse: 1.98602
[900]	valid_0's rmse: 1.95882
[1000]	valid_0's rmse: 1.93476
[1100]	valid_0's rmse: 1.9123
[1200]	valid_0's rmse: 1.89185
[1300]	valid_0's rmse: 1.87121
[1400]	valid_0's rmse: 1.852
[1500]	valid_0's rmse: 1.83419
Did not meet early stopping. Best iteration is:
[1500]	valid_0's rmse: 1.83419
save model_0_0


store 1.0

Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 1.7055
[200]	valid_0's rmse: 1.66828
[300]	valid_0's rmse: 1.63876
[400]	valid_0's rmse: 1.61261
[500]	valid_0's rmse: 1.58956
[600]	valid_0's rmse: 1.56725
[700]	valid_0's rmse: 1.54724
[800]	valid_0's rmse: 1.52737
[900]	valid_0's rmse: 1.5

0

In [6]:
with open("../input/2-stats-id/1_X_TX.pkl", "rb") as x1:
    X1 = x1.read()

with open("../input/2-2-stats-id/2_X_TX.pkl", "rb") as x2:
    X2 = x2.read()
    
X_tx_1 = pickle.loads(X1)
X_tx_2 = pickle.loads(X2)

del x1,x2, X1, X2
gc.collect()


X_tx = [X_tx_1, X_tx_2]

del X_tx_1, X_tx_2 
gc.collect()

0

In [7]:
i=0

for xi in X_tx:
    
    print(xi.shape)
    
    STORES = np.unique(xi[:,3])
    print('')
    print('number of STORES')
    print(STORES)
    print('')


    for store in STORES:

        print('')
        print('store '+str(store))
        print('')
        xi_store = xi[xi[:,3]==store]

        #18 -day
        #19 -month
        #20 - year

        mask_XYtrain = np.logical_or(xi_store[:,21]<=24,xi_store[:,22]<=4,xi_store[:,23]<=2016)

        # <= '2016-03-27' ~
        XY_train =  xi_store#[mask_XYtrain]
        XY_val =  xi_store[~mask_XYtrain]

        XY_train = np.delete(XY_train,[3,4],1)
        XY_val = np.delete(XY_val,[3,4],1)

        X_train = XY_train[:,:-1]
        y_train = XY_train[:,-1]
        X_val = XY_val[:,:-1]
        y_val = XY_val[:,-1]

        del xi_store, XY_train, XY_val
        gc.collect()

        train_set = lgb.Dataset(X_train, y_train.reshape(-1))
        val_set = lgb.Dataset(X_val, y_val.reshape(-1))

        seed_everything(SEED)
        model_tx = lgb.train(params, train_set,
                           valid_sets = [val_set],
#                           num_boost_round = 2500,
#                           early_stopping_rounds = 60,
                          verbose_eval = 100)

        model_tx.save_model('model_tx_'+str(i)+'.txt')
        print('save model_'+str(i))
        print('')

        del train_set, val_set, model_tx
        gc.collect()

        i+=1
        
    del xi
    gc.collect()
    
del X_tx
gc.collect()

(10573932, 65)

number of STORES
[0. 1.]


store 0.0

Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 1.84278
[200]	valid_0's rmse: 1.79778
[300]	valid_0's rmse: 1.76137
[400]	valid_0's rmse: 1.73303
[500]	valid_0's rmse: 1.70461
[600]	valid_0's rmse: 1.68046
[700]	valid_0's rmse: 1.65649
[800]	valid_0's rmse: 1.63223
[900]	valid_0's rmse: 1.61158
[1000]	valid_0's rmse: 1.59242
[1100]	valid_0's rmse: 1.5714
[1200]	valid_0's rmse: 1.55265
[1300]	valid_0's rmse: 1.5355
[1400]	valid_0's rmse: 1.51822
[1500]	valid_0's rmse: 1.50261
Did not meet early stopping. Best iteration is:
[1500]	valid_0's rmse: 1.50261
save model_0


store 1.0

Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 2.1735
[200]	valid_0's rmse: 2.12133
[300]	valid_0's rmse: 2.07263
[400]	valid_0's rmse: 2.03386
[500]	valid_0's rmse: 1.99607
[600]	valid_0's rmse: 1.96509
[700]	valid_0's rmse: 1.93397
[800]	valid_0's rmse: 1.9045
[900]	valid_0's rmse: 1.878

0

In [8]:
with open("../input/3-stats-id/1_X_WI.pkl", "rb") as x1:
    X1 = x1.read()

with open("../input/3-2-stats-id/2_X_WI.pkl", "rb") as x2:
    X2 = x2.read()
    
X_wi_1 = pickle.loads(X1)
X_wi_2 = pickle.loads(X2)

del x1,x2, X1, X2
gc.collect()


X_wi = [X_wi_1, X_wi_2]

del X_wi_1, X_wi_2 
gc.collect()

0

In [9]:
i=0

for xi in X_wi:
    
    print(xi.shape)
    
    STORES = np.unique(xi[:,3])
    print('')
    print('number of STORES')
    print(STORES)
    print('')

    for store in STORES:

        print('')
        print('store '+str(store))
        print('')
        xi_store = xi[xi[:,3]==store]

        #18 -day
        #19 -month
        #20 - year

        mask_XYtrain = np.logical_or(xi_store[:,21]<=24,xi_store[:,22]<=4,xi_store[:,23]<=2016)

        # <= '2016-03-27' ~
        XY_train =  xi_store#[mask_XYtrain]
        XY_val =  xi_store[~mask_XYtrain]

        XY_train = np.delete(XY_train,[3,4],1)
        XY_val = np.delete(XY_val,[3,4],1)

        X_train = XY_train[:,:-1]
        y_train = XY_train[:,-1]
        X_val = XY_val[:,:-1]
        y_val = XY_val[:,-1]

        del xi_store, XY_train, XY_val
        gc.collect()

        train_set = lgb.Dataset(X_train, y_train.reshape(-1))
        val_set = lgb.Dataset(X_val, y_val.reshape(-1))

        seed_everything(SEED)
        model_wi = lgb.train(params, train_set,
                           valid_sets = [val_set],
#                           num_boost_round = 2500,
#                           early_stopping_rounds = 60,
                          verbose_eval = 100)

        model_wi.save_model('model_wi_'+str(i)+'.txt')
        print('save model_'+str(i))
        print('')

        del train_set, val_set, model_wi
        gc.collect()

        i+=1
        
    del xi
    gc.collect()
    
del X_wi
gc.collect()

(10573932, 65)

number of STORES
[0. 1.]


store 0.0

Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 1.49502
[200]	valid_0's rmse: 1.46289
[300]	valid_0's rmse: 1.43805
[400]	valid_0's rmse: 1.41597
[500]	valid_0's rmse: 1.39618
[600]	valid_0's rmse: 1.37789
[700]	valid_0's rmse: 1.36077
[800]	valid_0's rmse: 1.34455
[900]	valid_0's rmse: 1.32984
[1000]	valid_0's rmse: 1.31588
[1100]	valid_0's rmse: 1.30271
[1200]	valid_0's rmse: 1.28993
[1300]	valid_0's rmse: 1.27789
[1400]	valid_0's rmse: 1.26626
[1500]	valid_0's rmse: 1.25496
Did not meet early stopping. Best iteration is:
[1500]	valid_0's rmse: 1.25496
save model_0


store 1.0

Training until validation scores don't improve for 30 rounds
[100]	valid_0's rmse: 1.80286
[200]	valid_0's rmse: 1.7587
[300]	valid_0's rmse: 1.72499
[400]	valid_0's rmse: 1.69416
[500]	valid_0's rmse: 1.66458
[600]	valid_0's rmse: 1.63867
[700]	valid_0's rmse: 1.61522
[800]	valid_0's rmse: 1.59348
[900]	valid_0's rmse: 1.

0

## ALL

In [10]:
# import pickle

# with open("../input/1-stats-id/1_X_CA.pkl", "rb") as x1:
#     X1 = x1.read()
    
# with open("../input/1-2-stats-id/2_X_CA.pkl", "rb") as x1_1:
#     X1_1 = x1_1.read()
    
# del x1, x1_1 
# gc.collect()

# X_ca_1 = pickle.loads(X1)
# X_ca_2 = pickle.loads(X1_1)

# X_ca = [X_ca_1, X_ca_2]

# del X1, X1_1, X_ca_1, X_ca_2 
# gc.collect()

In [11]:
# i=0

# for xi in X_ca:
    
#     print(xi.shape)
    
#     STORES = np.unique(xi[:,3])
#     print('')
#     print('number of STORES')
#     print(STORES)
#     print('')

#     for store in STORES:

#         print('')
#         print('store '+str(i))
#         print('')
#         xi_store = xi[xi[:,3]==store]
        
#         print(xi_store[:2,:4])

#         #18 -day
#         #19 -month
#         #20 - year

#         mask_XYtrain = np.logical_or(xi_store[:,18]<=27,xi_store[:,19]<=3,xi_store[:,20]<=2016)

#         # <= '2016-03-27' ~
#         XY_train =  xi_store#[mask_XYtrain]
#         XY_val =  xi_store[~mask_XYtrain]

#         X_train = XY_train[:,:-1]
#         y_train = XY_train[:,-1]
#         X_val = XY_val[:,:-1]
#         y_val = XY_val[:,-1]

#         del xi_store, XY_train, XY_val
#         gc.collect()

#         train_set = lgb.Dataset(X_train, y_train.reshape(-1))
#         val_set = lgb.Dataset(X_val, y_val.reshape(-1))
        
#         if i==0:
            
#             print(i)
#             seed_everything(SEED)
#             model_ca = lgb.train(params, train_set,
#                                valid_sets = [val_set],
# #                               num_boost_round = 2000,
# #                               early_stopping_rounds = 60,
#                               verbose_eval = 100)
            
    
#         else:
            
#             print(i)
            
#             seed_everything(SEED)
#             model_ca = lgb.train(params, train_set,
#                                valid_sets = [val_set],
# #                               num_boost_round = 2000,
# #                               early_stopping_rounds = 60,
#                               init_model = model_ca,
#                               verbose_eval = 100)

#         del train_set, val_set
#         gc.collect()

#         i+=1
        
#     del xi
#     gc.collect()

# del X_ca
# gc.collect()

In [12]:
# with open("../input/2-stats-id/1_X_TX.pkl", "rb") as x1:
#     X1 = x1.read()

# with open("../input/2-2-stats-id/2_X_TX.pkl", "rb") as x2:
#     X2 = x2.read()
    
# X_tx_1 = pickle.loads(X1)
# X_tx_2 = pickle.loads(X2)

# del x1,x2, X1, X2
# gc.collect()


# X_tx = [X_tx_1, X_tx_2]

# del X_tx_1, X_tx_2 
# gc.collect()

In [13]:
# i=0

# for xi in X_tx:
    
#     print(xi.shape)
    
#     STORES = np.unique(xi[:,3])
#     print('')
#     print('number of STORES')
#     print(STORES)
#     print('')


#     for store in STORES:

#         print('')
#         print('store '+str(i))
#         print('')
#         xi_store = xi[xi[:,3]==store]

#         #18 -day
#         #19 -month
#         #20 - year

#         mask_XYtrain = np.logical_or(xi_store[:,18]<=27,xi_store[:,19]<=3,xi_store[:,20]<=2016)

#         # <= '2016-03-27' ~
#         XY_train =  xi_store#[mask_XYtrain]
#         XY_val =  xi_store[~mask_XYtrain]

#         X_train = XY_train[:,:-1]
#         y_train = XY_train[:,-1]
#         X_val = XY_val[:,:-1]
#         y_val = XY_val[:,-1]

#         del xi_store, XY_train, XY_val
#         gc.collect()

#         train_set = lgb.Dataset(X_train, y_train.reshape(-1))
#         val_set = lgb.Dataset(X_val, y_val.reshape(-1))

#         if i==0:
            
#             seed_everything(SEED)
#             model_tx = lgb.train(params, train_set,
#                                valid_sets = [val_set],
# #                               num_boost_round = 2000,
# #                               early_stopping_rounds = 60,
#                             init_model = model_ca,
#                               verbose_eval = 100)
#             del model_ca
#             gc.collect()
            
#         else:
            
#             seed_everything(SEED)
#             model_tx = lgb.train(params, train_set,
#                                valid_sets = [val_set],
# #                               num_boost_round = 2000,
# #                               early_stopping_rounds = 60,
#                               init_model = model_tx,
#                               verbose_eval = 100)
        

#         del train_set, val_set
#         gc.collect()

#         i+=1
        
#     del xi
#     gc.collect()
    
# del X_tx
# gc.collect()

In [14]:
# with open("../input/3-stats-id/1_X_WI.pkl", "rb") as x1:
#     X1 = x1.read()

# with open("../input/3-2-stats-id/2_X_WI.pkl", "rb") as x2:
#     X2 = x2.read()
    
# X_wi_1 = pickle.loads(X1)
# X_wi_2 = pickle.loads(X2)

# del x1,x2, X1, X2
# gc.collect()


# X_wi = [X_wi_1, X_wi_2]

# del X_wi_1, X_wi_2 
# gc.collect()

In [15]:
# i=0

# for xi in X_wi:
    
#     print(xi.shape)
    
#     STORES = np.unique(xi[:,3])
#     print('')
#     print('number of STORES')
#     print(STORES)
#     print('')

#     for store in STORES:

#         print('')
#         print('store '+str(i))
#         print('')
#         xi_store = xi[xi[:,3]==store]

#         #18 -day
#         #19 -month
#         #20 - year

#         mask_XYtrain = np.logical_or(xi_store[:,18]<=27,xi_store[:,19]<=3,xi_store[:,20]<=2016)

#         # <= '2016-03-27' ~
#         XY_train =  xi_store#[mask_XYtrain]
#         XY_val =  xi_store[~mask_XYtrain]

#         X_train = XY_train[:,:-1]
#         y_train = XY_train[:,-1]
#         X_val = XY_val[:,:-1]
#         y_val = XY_val[:,-1]

#         del xi_store, XY_train, XY_val
#         gc.collect()

#         train_set = lgb.Dataset(X_train, y_train.reshape(-1))
#         val_set = lgb.Dataset(X_val, y_val.reshape(-1))
        
#         if i==0:

#             seed_everything(SEED)
#             model_wi = lgb.train(params, train_set,
#                                valid_sets = [val_set],
#                               init_model = model_tx,
# #                               num_boost_round = 2000,  
# #                               early_stopping_rounds = 60,
#                               verbose_eval = 100)
#             del model_tx
#             gc.collect()
            
#         else:
            
#             seed_everything(SEED)
#             model_wi = lgb.train(params, train_set,
#                                valid_sets = [val_set],
#                               init_model = model_wi,
# #                               num_boost_round = 2000,   
# #                               early_stopping_rounds = 60,
#                               verbose_eval = 100)
            
#         if i ==2:
            
#             model_wi.save_model('model.txt')
#             print('save model ')
#             print('')

#         del train_set, val_set
#         gc.collect()

#         i+=1
        
#     del xi
#     gc.collect()
    
# del X_wi
# gc.collect()

In [16]:
# fig, ax = plt.subplots(figsize=(10, 7))
# lgb.plot_importance(model, max_num_features=10, ax=ax)
# plt.title("LightGBM - Feature Importance");